In [ ]:
from universal_chain import UniversalChain

chain = UniversalChain()
print(chain.run("https://www.youtube.com/watch?v=afWBnxWQDKk"))

In [1]:
from YouTubeLoader.youtube import url_to_subtitles

print(url_to_subtitles("https://www.youtube.com/watch?v=afWBnxWQDKk"))

Downloaded audio: .cache/afWBnxWQDKk/afWBnxWQDKk.mp3
Available subtitles: {'a.en': <Caption lang="English (auto-generated)" code="a.en">}
File already exists. Forcing download of https://fal.media/files/zebra/UJLntD6kbclNJQEIFVM_I.mpga to /tmp/tmpijl5jtcx/UJLntD6kbclNJQEIFVM_I.mpga
File already exists. Forcing download of https://fal.media/files/zebra/UJLntD6kbclNJQEIFVM_I.mpga to /tmp/tmpijl5jtcx/UJLntD6kbclNJQEIFVM_I.mpga
File already exists. Forcing download of https://fal.media/files/zebra/UJLntD6kbclNJQEIFVM_I.mpga to /tmp/tmpijl5jtcx/UJLntD6kbclNJQEIFVM_I.mpga
File already exists. Forcing download of https://fal.media/files/zebra/UJLntD6kbclNJQEIFVM_I.mpga to /tmp/tmpijl5jtcx/UJLntD6kbclNJQEIFVM_I.mpga
Downloaded input audio in 1.79353997297585 seconds
File already exists. Forcing download of https://fal.media/files/zebra/UJLntD6kbclNJQEIFVM_I.mpga to /tmp/tmpijl5jtcx/UJLntD6kbclNJQEIFVM_I.mpga
Downloaded input audio in 1.79353997297585 seconds
File already exists. Forcing downlo

In [1]:
import os
import pandas as pd
import sqlite3
from pandasai import SmartDataframe
from pandas import read_sql_query, read_sql_table
from dotenv import load_dotenv

load_dotenv()


with sqlite3.connect("databases/Chinook.db") as connection:
    tables = list(read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", connection)["name"])
    output = {table: read_sql_query(f"SELECT * from {table}", connection) for table in tables}

    df = SmartDataframe(output["Album"])
    df.chat("What is the average number of milliseconds in the database?")

<string>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


In [5]:
out

{'Album':      AlbumId                                              Title  ArtistId
 0          1              For Those About To Rock We Salute You         1
 1          2                                  Balls to the Wall         2
 2          3                                  Restless and Wild         2
 3          4                                  Let There Be Rock         1
 4          5                                           Big Ones         3
 ..       ...                                                ...       ...
 342      343                             Respighi:Pines of Rome       226
 343      344  Schubert: The Late String Quartets & String Qu...       272
 344      345                                Monteverdi: L'Orfeo       273
 345      346                              Mozart: Chamber Music       274
 346      347  Koyaanisqatsi (Soundtrack from the Motion Pict...       275
 
 [347 rows x 3 columns],
 'Artist':      ArtistId                                        

In [20]:
import re

from langchain import hub
from langchain.agents.agent import AgentExecutor
from langchain.agents.react.agent import create_react_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.chat_models.base import ChatOpenAI

sql_prompt = hub.pull("langchain-ai/sql-agent-system-prompt")
react_prompt = hub.pull("hwchase17/react")
combined_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sql_prompt.messages[0].prompt.template + "\n" + react_prompt.template),
    ]
)
print(combined_prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'dialect', 'input', 'tool_names', 'tools', 'top_k'], input_types={}, partial_variables={}, template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import re
from langchain.document_loaders import UnstructuredURLLoader
from langchain_core.documents.base import Document
from typing import List

url = "https://lilianweng.github.io/posts/2023-06-23-agent/"


def webloader(url: str) -> List[Document]:
    """Load the content of a website from url to text."""
    docs = WebBaseLoader(url).load()
    docs = [re.sub(r"\n{3,}", r"\n\n", doc.page_content) for doc in docs]
    return docs


print(webloader(url))

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import re
from langchain.document_loaders import UnstructuredURLLoader

url = "https://lilianweng.github.io/posts/2023-06-23-agent/"


print(UnstructuredURLLoader(urls=[url]).load())

In [ ]:
from docling.document_converter import DocumentConverter

source = "https://lilianweng.github.io/posts/2023-06-23-agent/"  # PDF path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())

In [1]:
from TextToSQL.react import text_to_sql_react
from TextToSQL.simple import text_to_sql


questions = ["What is the schema of the database"]

for i, question in enumerate(questions, 1):
    print(f"\nQuestion {i}:\n{question}")

    print(f"\nSimple Response {i}:")
    response = text_to_sql(question)
    print(response)

    print(f"\nReAct Response {i}:")
    response = text_to_sql_react(question)
    print(response)

    if i < len(questions):
        print("---")


Question 1:
What is the schema of the database

Simple Response 1:
'NoneType' object has no attribute 'group'

ReAct Response 1:
[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f5c46aa3a40>), InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f5c46aa3a40>), ListSQLDatabaseTool(db=<langchain_communit

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage


response = {
    "messages": [
        HumanMessage(content="What is the schema", additional_kwargs={}, response_metadata={}, id="d2119e78-fa98-4a9d-9bcd-398a40f5fcca"),
        AIMessage(content="", additional_kwargs={"tool_calls": [{"id": "call_aq8v5FjcufSgTJwHIKw3E2uP", "function": {"arguments": "{}", "name": "sql_db_list_tables"}, "type": "function"}], "refusal": None}, response_metadata={"token_usage": {"completion_tokens": 12, "prompt_tokens": 550, "total_tokens": 562, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None}, id="run-adb765b4-b4c2-483d-8fa7-8d1483593fee-0", tool_calls=[{"name": "sql_db_list_tables", "args": {}, "id": "call_aq8v5FjcufSgTJwHIKw3E2uP", "type": "tool_call"}], usage_metadata={"input_tokens": 550, "output_tokens": 12, "total_tokens": 562, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}}),
        ToolMessage(content="Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track", name="sql_db_list_tables", id="63030397-92bf-4ca6-b15a-4d74992bc847", tool_call_id="call_aq8v5FjcufSgTJwHIKw3E2uP"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_zNxhybANPpeetfuzY5XVMUHF", "function": {"arguments": '{"table_names":"Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track"}', "name": "sql_db_schema"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 39, "prompt_tokens": 596, "total_tokens": 635, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-45c060f5-6fa0-4f42-9bd1-cb7bce6edde6-0",
            tool_calls=[{"name": "sql_db_schema", "args": {"table_names": "Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track"}, "id": "call_zNxhybANPpeetfuzY5XVMUHF", "type": "tool_call"}],
            usage_metadata={"input_tokens": 596, "output_tokens": 39, "total_tokens": 635, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(
            content='\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE "Artist" (\n\t"ArtistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("ArtistId")\n)\n\n/*\n3 rows from Artist table:\nArtistId\tName\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE "Customer" (\n\t"CustomerId" INTEGER NOT NULL, \n\t"FirstName" NVARCHAR(40) NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"Company" NVARCHAR(80), \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(40), \n\t"Country" NVARCHAR(40), \n\t"PostalCode" NVARCHAR(10), \n\t"Phone" NVARCHAR(24), \n\t"Fax" NVARCHAR(24), \n\t"Email" NVARCHAR(60) NOT NULL, \n\t"SupportRepId" INTEGER, \n\tPRIMARY KEY ("CustomerId"), \n\tFOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")\n)\n\n/*\n3 rows from Customer table:\nCustomerId\tFirstName\tLastName\tCompany\tAddress\tCity\tState\tCountry\tPostalCode\tPhone\tFax\tEmail\tSupportRepId\n1\tLuís\tGonçalves\tEmbraer - Empresa Brasileira de Aeronáutica S.A.\tAv. Brigadeiro Faria Lima, 2170\tSão José dos Campos\tSP\tBrazil\t12227-000\t+55 (12) 3923-5555\t+55 (12) 3923-5566\tluisg@embraer.com.br\t3\n2\tLeonie\tKöhler\tNone\tTheodor-Heuss-Straße 34\tStuttgart\tNone\tGermany\t70174\t+49 0711 2842222\tNone\tleonekohler@surfeu.de\t5\n3\tFrançois\tTremblay\tNone\t1498 rue Bélanger\tMontréal\tQC\tCanada\tH2G 1A7\t+1 (514) 721-4711\tNone\tftremblay@gmail.com\t3\n*/\n\n\nCREATE TABLE "Employee" (\n\t"EmployeeId" INTEGER NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"FirstName" NVARCHAR(20) NOT NULL, \n\t"Title" NVARCHAR(30), \n\t"ReportsTo" INTEGER, \n\t"BirthDate" DATETIME, \n\t"HireDate" DATETIME, \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(40), \n\t"Country" NVARCHAR(40), \n\t"PostalCode" NVARCHAR(10), \n\t"Phone" NVARCHAR(24), \n\t"Fax" NVARCHAR(24), \n\t"Email" NVARCHAR(60), \n\tPRIMARY KEY ("EmployeeId"), \n\tFOREIGN KEY("ReportsTo") REFERENCES "Employee" ("EmployeeId")\n)\n\n/*\n3 rows from Employee table:\nEmployeeId\tLastName\tFirstName\tTitle\tReportsTo\tBirthDate\tHireDate\tAddress\tCity\tState\tCountry\tPostalCode\tPhone\tFax\tEmail\n1\tAdams\tAndrew\tGeneral Manager\tNone\t1962-02-18 00:00:00\t2002-08-14 00:00:00\t11120 Jasper Ave NW\tEdmonton\tAB\tCanada\tT5K 2N1\t+1 (780) 428-9482\t+1 (780) 428-3457\tandrew@chinookcorp.com\n2\tEdwards\tNancy\tSales Manager\t1\t1958-12-08 00:00:00\t2002-05-01 00:00:00\t825 8 Ave SW\tCalgary\tAB\tCanada\tT2P 2T3\t+1 (403) 262-3443\t+1 (403) 262-3322\tnancy@chinookcorp.com\n3\tPeacock\tJane\tSales Support Agent\t2\t1973-08-29 00:00:00\t2002-04-01 00:00:00\t1111 6 Ave SW\tCalgary\tAB\tCanada\tT2P 5M5\t+1 (403) 262-3443\t+1 (403) 262-6712\tjane@chinookcorp.com\n*/\n\n\nCREATE TABLE "Genre" (\n\t"GenreId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("GenreId")\n)\n\n/*\n3 rows from Genre table:\nGenreId\tName\n1\tRock\n2\tJazz\n3\tMetal\n*/\n\n\nCREATE TABLE "Invoice" (\n\t"InvoiceId" INTEGER NOT NULL, \n\t"CustomerId" INTEGER NOT NULL, \n\t"InvoiceDate" DATETIME NOT NULL, \n\t"BillingAddress" NVARCHAR(70), \n\t"BillingCity" NVARCHAR(40), \n\t"BillingState" NVARCHAR(40), \n\t"BillingCountry" NVARCHAR(40), \n\t"BillingPostalCode" NVARCHAR(10), \n\t"Total" NUMERIC(10, 2) NOT NULL, \n\tPRIMARY KEY ("InvoiceId"), \n\tFOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")\n)\n\n/*\n3 rows from Invoice table:\nInvoiceId\tCustomerId\tInvoiceDate\tBillingAddress\tBillingCity\tBillingState\tBillingCountry\tBillingPostalCode\tTotal\n1\t2\t2021-01-01 00:00:00\tTheodor-Heuss-Straße 34\tStuttgart\tNone\tGermany\t70174\t1.98\n2\t4\t2021-01-02 00:00:00\tUllevålsveien 14\tOslo\tNone\tNorway\t0171\t3.96\n3\t8\t2021-01-03 00:00:00\tGrétrystraat 63\tBrussels\tNone\tBelgium\t1000\t5.94\n*/\n\n\nCREATE TABLE "InvoiceLine" (\n\t"InvoiceLineId" INTEGER NOT NULL, \n\t"InvoiceId" INTEGER NOT NULL, \n\t"TrackId" INTEGER NOT NULL, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\t"Quantity" INTEGER NOT NULL, \n\tPRIMARY KEY ("InvoiceLineId"), \n\tFOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), \n\tFOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")\n)\n\n/*\n3 rows from InvoiceLine table:\nInvoiceLineId\tInvoiceId\tTrackId\tUnitPrice\tQuantity\n1\t1\t2\t0.99\t1\n2\t1\t4\t0.99\t1\n3\t2\t6\t0.99\t1\n*/\n\n\nCREATE TABLE "MediaType" (\n\t"MediaTypeId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("MediaTypeId")\n)\n\n/*\n3 rows from MediaType table:\nMediaTypeId\tName\n1\tMPEG audio file\n2\tProtected AAC audio file\n3\tProtected MPEG-4 video file\n*/\n\n\nCREATE TABLE "Playlist" (\n\t"PlaylistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("PlaylistId")\n)\n\n/*\n3 rows from Playlist table:\nPlaylistId\tName\n1\tMusic\n2\tMovies\n3\tTV Shows\n*/\n\n\nCREATE TABLE "PlaylistTrack" (\n\t"PlaylistId" INTEGER NOT NULL, \n\t"TrackId" INTEGER NOT NULL, \n\tPRIMARY KEY ("PlaylistId", "TrackId"), \n\tFOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), \n\tFOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")\n)\n\n/*\n3 rows from PlaylistTrack table:\nPlaylistId\tTrackId\n1\t3402\n1\t3389\n1\t3390\n*/\n\n\nCREATE TABLE "Track" (\n\t"TrackId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(200) NOT NULL, \n\t"AlbumId" INTEGER, \n\t"MediaTypeId" INTEGER NOT NULL, \n\t"GenreId" INTEGER, \n\t"Composer" NVARCHAR(220), \n\t"Milliseconds" INTEGER NOT NULL, \n\t"Bytes" INTEGER, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\tPRIMARY KEY ("TrackId"), \n\tFOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), \n\tFOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), \n\tFOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")\n)\n\n/*\n3 rows from Track table:\nTrackId\tName\tAlbumId\tMediaTypeId\tGenreId\tComposer\tMilliseconds\tBytes\tUnitPrice\n1\tFor Those About To Rock (We Salute You)\t1\t1\t1\tAngus Young, Malcolm Young, Brian Johnson\t343719\t11170334\t0.99\n2\tBalls to the Wall\t2\t2\t1\tU. Dirkschneider, W. Hoffmann, H. Frank, P. Baltes, S. Kaufmann, G. Hoffmann\t342562\t5510424\t0.99\n3\tFast As a Shark\t3\t2\t1\tF. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman\t230619\t3990994\t0.99\n*/',
            name="sql_db_schema",
            id="87eceb79-701d-464d-8863-a82fb34923f3",
            tool_call_id="call_zNxhybANPpeetfuzY5XVMUHF",
        ),
        AIMessage(
            content="The database contains the following tables along with their schemas:\n\n1. **Album**\n   - **AlbumId**: INTEGER (Primary Key)\n   - **Title**: NVARCHAR(160) (Not Null)\n   - **ArtistId**: INTEGER (Not Null, Foreign Key referencing Artist)\n\n2. **Artist**\n   - **ArtistId**: INTEGER (Primary Key)\n   - **Name**: NVARCHAR(120)\n\n3. **Customer**\n   - **CustomerId**: INTEGER (Primary Key)\n   - **FirstName**: NVARCHAR(40) (Not Null)\n   - **LastName**: NVARCHAR(20) (Not Null)\n   - **Company**: NVARCHAR(80)\n   - **Address**: NVARCHAR(70)\n   - **City**: NVARCHAR(40)\n   - **State**: NVARCHAR(40)\n   - **Country**: NVARCHAR(40)\n   - **PostalCode**: NVARCHAR(10)\n   - **Phone**: NVARCHAR(24)\n   - **Fax**: NVARCHAR(24)\n   - **Email**: NVARCHAR(60) (Not Null)\n   - **SupportRepId**: INTEGER (Foreign Key referencing Employee)\n\n4. **Employee**\n   - **EmployeeId**: INTEGER (Primary Key)\n   - **LastName**: NVARCHAR(20) (Not Null)\n   - **FirstName**: NVARCHAR(20) (Not Null)\n   - **Title**: NVARCHAR(30)\n   - **ReportsTo**: INTEGER (Foreign Key referencing Employee)\n   - **BirthDate**: DATETIME\n   - **HireDate**: DATETIME\n   - **Address**: NVARCHAR(70)\n   - **City**: NVARCHAR(40)\n   - **State**: NVARCHAR(40)\n   - **Country**: NVARCHAR(40)\n   - **PostalCode**: NVARCHAR(10)\n   - **Phone**: NVARCHAR(24)\n   - **Fax**: NVARCHAR(24)\n   - **Email**: NVARCHAR(60)\n\n5. **Genre**\n   - **GenreId**: INTEGER (Primary Key)\n   - **Name**: NVARCHAR(120)\n\n6. **Invoice**\n   - **InvoiceId**: INTEGER (Primary Key)\n   - **CustomerId**: INTEGER (Not Null, Foreign Key referencing Customer)\n   - **InvoiceDate**: DATETIME (Not Null)\n   - **BillingAddress**: NVARCHAR(70)\n   - **BillingCity**: NVARCHAR(40)\n   - **BillingState**: NVARCHAR(40)\n   - **BillingCountry**: NVARCHAR(40)\n   - **BillingPostalCode**: NVARCHAR(10)\n   - **Total**: NUMERIC(10, 2) (Not Null)\n\n7. **InvoiceLine**\n   - **InvoiceLineId**: INTEGER (Primary Key)\n   - **InvoiceId**: INTEGER (Not Null, Foreign Key referencing Invoice)\n   - **TrackId**: INTEGER (Not Null, Foreign Key referencing Track)\n   - **UnitPrice**: NUMERIC(10, 2) (Not Null)\n   - **Quantity**: INTEGER (Not Null)\n\n8. **MediaType**\n   - **MediaTypeId**: INTEGER (Primary Key)\n   - **Name**: NVARCHAR(120)\n\n9. **Playlist**\n   - **PlaylistId**: INTEGER (Primary Key)\n   - **Name**: NVARCHAR(120)\n\n10. **PlaylistTrack**\n    - **PlaylistId**: INTEGER (Not Null, Foreign Key referencing Playlist)\n    - **TrackId**: INTEGER (Not Null, Foreign Key referencing Track)\n    - (Primary Key is a composite of PlaylistId and TrackId)\n\n11. **Track**\n    - **TrackId**: INTEGER (Primary Key)\n    - **Name**: NVARCHAR(200) (Not Null)\n    - **AlbumId**: INTEGER (Foreign Key referencing Album)\n    - **MediaTypeId**: INTEGER (Not Null, Foreign Key referencing MediaType)\n    - **GenreId**: INTEGER (Foreign Key referencing Genre)\n    - **Composer**: NVARCHAR(220)\n    - **Milliseconds**: INTEGER (Not Null)\n    - **Bytes**: INTEGER\n    - **UnitPrice**: NUMERIC(10, 2) (Not Null)\n\nThis schema provides a comprehensive overview of the structure of the database and the relationships between the tables.",
            additional_kwargs={"refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 937, "prompt_tokens": 2790, "total_tokens": 3727, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 2688}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "stop", "logprobs": None},
            id="run-9751d5be-1ed7-483f-8e50-cfff3bc11ff6-0",
            usage_metadata={"input_tokens": 2790, "output_tokens": 937, "total_tokens": 3727, "input_token_details": {"audio": 0, "cache_read": 2688}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
    ]
}

In [3]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage


reversed_messages = reversed(response["messages"])
answer_msg = next((msg for msg in reversed_messages if isinstance(msg, AIMessage)), None)
answer = answer_msg.content if answer_msg else ""
print(answer)

data_msg = next((msg for msg in reversed_messages if isinstance(msg, ToolMessage) and msg.name == "sql_db_query"), None)
data = data_msg.content if data_msg else ""
query_call_id = data_msg.tool_call_id if data_msg else ""
print(data)
print(query_call_id)

# Find the query execution matching the query call id
query_msg = next((msg for msg in reversed_messages if isinstance(msg, AIMessage) and msg.tool_calls and any(call["id"] == query_call_id for call in msg.tool_calls)), None)
query = query_msg.tool_calls[0]["args"]["query"] if data_msg else ""
print(query)

table_name = re.search(r"FROM\s+(\w+)", query, re.IGNORECASE).group(1) if query else ""
print(table_name)

The database contains the following tables along with their schemas:

1. **Album**
   - **AlbumId**: INTEGER (Primary Key)
   - **Title**: NVARCHAR(160) (Not Null)
   - **ArtistId**: INTEGER (Not Null, Foreign Key referencing Artist)

2. **Artist**
   - **ArtistId**: INTEGER (Primary Key)
   - **Name**: NVARCHAR(120)

3. **Customer**
   - **CustomerId**: INTEGER (Primary Key)
   - **FirstName**: NVARCHAR(40) (Not Null)
   - **LastName**: NVARCHAR(20) (Not Null)
   - **Company**: NVARCHAR(80)
   - **Address**: NVARCHAR(70)
   - **City**: NVARCHAR(40)
   - **State**: NVARCHAR(40)
   - **Country**: NVARCHAR(40)
   - **PostalCode**: NVARCHAR(10)
   - **Phone**: NVARCHAR(24)
   - **Fax**: NVARCHAR(24)
   - **Email**: NVARCHAR(60) (Not Null)
   - **SupportRepId**: INTEGER (Foreign Key referencing Employee)

4. **Employee**
   - **EmployeeId**: INTEGER (Primary Key)
   - **LastName**: NVARCHAR(20) (Not Null)
   - **FirstName**: NVARCHAR(20) (Not Null)
   - **Title**: NVARCHAR(30)
   - **Repor

In [21]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

response = {
    "messages": [
        HumanMessage(content="What are the 10 most expensive albums", additional_kwargs={}, response_metadata={}, id="08c293f3-c850-4861-bbae-ab333ea61dfc"),
        AIMessage(content="", additional_kwargs={"tool_calls": [{"id": "call_ujwhC8lXKlibq8mdGbL3hOQo", "function": {"arguments": "{}", "name": "sql_db_list_tables"}, "type": "function"}], "refusal": None}, response_metadata={"token_usage": {"completion_tokens": 12, "prompt_tokens": 554, "total_tokens": 566, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None}, id="run-00fb77df-ea09-45a7-9c04-075dc6692af4-0", tool_calls=[{"name": "sql_db_list_tables", "args": {}, "id": "call_ujwhC8lXKlibq8mdGbL3hOQo", "type": "tool_call"}], usage_metadata={"input_tokens": 554, "output_tokens": 12, "total_tokens": 566, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}}),
        ToolMessage(content="Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track", name="sql_db_list_tables", id="2639cd20-79be-4145-8c28-fd7c98c11998", tool_call_id="call_ujwhC8lXKlibq8mdGbL3hOQo"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_FkDZxE17O28rtluSfsePh1vy", "function": {"arguments": '{"table_names": "Album"}', "name": "sql_db_schema"}, "type": "function"}, {"id": "call_8br8wAw8mKZJrolWl8xnI3yn", "function": {"arguments": '{"table_names": "InvoiceLine"}', "name": "sql_db_schema"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 49, "prompt_tokens": 600, "total_tokens": 649, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-3b412371-8877-42c3-a5b3-0be49942be52-0",
            tool_calls=[{"name": "sql_db_schema", "args": {"table_names": "Album"}, "id": "call_FkDZxE17O28rtluSfsePh1vy", "type": "tool_call"}, {"name": "sql_db_schema", "args": {"table_names": "InvoiceLine"}, "id": "call_8br8wAw8mKZJrolWl8xnI3yn", "type": "tool_call"}],
            usage_metadata={"input_tokens": 600, "output_tokens": 49, "total_tokens": 649, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content='\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/', name="sql_db_schema", id="b4d20b87-a5d6-4fa7-b586-6eef045a68d4", tool_call_id="call_FkDZxE17O28rtluSfsePh1vy"),
        ToolMessage(content='\nCREATE TABLE "InvoiceLine" (\n\t"InvoiceLineId" INTEGER NOT NULL, \n\t"InvoiceId" INTEGER NOT NULL, \n\t"TrackId" INTEGER NOT NULL, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\t"Quantity" INTEGER NOT NULL, \n\tPRIMARY KEY ("InvoiceLineId"), \n\tFOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), \n\tFOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")\n)\n\n/*\n3 rows from InvoiceLine table:\nInvoiceLineId\tInvoiceId\tTrackId\tUnitPrice\tQuantity\n1\t1\t2\t0.99\t1\n2\t1\t4\t0.99\t1\n3\t2\t6\t0.99\t1\n*/', name="sql_db_schema", id="44e0561a-fa3a-4454-a33d-ebd433e53ab8", tool_call_id="call_8br8wAw8mKZJrolWl8xnI3yn"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_T8fcCUSzvSrq1xkeiMnW1SAp", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 980, "total_tokens": 1054, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-084ef297-ca7b-41b8-9414-dff3295042fd-0",
            tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_T8fcCUSzvSrq1xkeiMnW1SAp", "type": "tool_call"}],
            usage_metadata={"input_tokens": 980, "output_tokens": 74, "total_tokens": 1054, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="728441e9-da64-4429-aaaa-283e8fad790f", tool_call_id="call_T8fcCUSzvSrq1xkeiMnW1SAp"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_An4AdvOZFahBfIs0bPr2ilEL", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 1123, "total_tokens": 1197, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-8dd504f0-5099-45a5-85a9-d4e493b2c6be-0",
            tool_calls=[{"name": "sql_db_query", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_An4AdvOZFahBfIs0bPr2ilEL", "type": "tool_call"}],
            usage_metadata={"input_tokens": 1123, "output_tokens": 74, "total_tokens": 1197, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content="Error: (sqlite3.OperationalError) no such column: il.AlbumId\n[SQL: SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)", name="sql_db_query", id="93c01a0d-4938-4e83-8923-622829d9f28c", tool_call_id="call_An4AdvOZFahBfIs0bPr2ilEL"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_3Zhdw8s6eUKZdqTSFJPdaday", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 1304, "total_tokens": 1378, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1024}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-2587b203-5208-4361-93bd-b184ae9da2d4-0",
            tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_3Zhdw8s6eUKZdqTSFJPdaday", "type": "tool_call"}],
            usage_metadata={"input_tokens": 1304, "output_tokens": 74, "total_tokens": 1378, "input_token_details": {"audio": 0, "cache_read": 1024}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="e7a0ba29-540e-44b4-b15b-8a518db40a0e", tool_call_id="call_3Zhdw8s6eUKZdqTSFJPdaday"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_vFl1B4aUK7JT3Hiup2VuvYEE", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 1447, "total_tokens": 1521, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1280}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-8636c183-a2e7-4655-9fba-0ebf41abfe4b-0",
            tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_vFl1B4aUK7JT3Hiup2VuvYEE", "type": "tool_call"}],
            usage_metadata={"input_tokens": 1447, "output_tokens": 74, "total_tokens": 1521, "input_token_details": {"audio": 0, "cache_read": 1280}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="da5403a8-0956-4342-93c1-346db7a38f8a", tool_call_id="call_vFl1B4aUK7JT3Hiup2VuvYEE"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_Uyoj19aI1O4Zdrv705wIGYEg", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 73, "prompt_tokens": 1590, "total_tokens": 1663, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1408}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-eec068c0-f712-4c08-9fb3-1619b1eb6120-0",
            tool_calls=[{"name": "sql_db_query", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_Uyoj19aI1O4Zdrv705wIGYEg", "type": "tool_call"}],
            usage_metadata={"input_tokens": 1590, "output_tokens": 73, "total_tokens": 1663, "input_token_details": {"audio": 0, "cache_read": 1408}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content="[('Balls to the Wall', 1.98), ('Warner 25 Anos', 1.98), ('Plays Metallica By Four Cellos', 1.98), ('The Best Of Buddy Guy - The Millenium Collection', 1.98), ('Carnaval 2001', 1.98), ('The Essential Miles Davis [Disc 1]', 1.98), ('The Battle Rages On', 1.98), ('Roda De Funk', 1.98), ('Demorou...', 1.98), ('Motley Crue Greatest Hits', 1.98)]", name="sql_db_query", id="70d28606-e327-48bb-927c-d48eff42b663", tool_call_id="call_Uyoj19aI1O4Zdrv705wIGYEg"),
        AIMessage(
            content="The 10 most expensive albums based on total sales are:\n\n1. **Balls to the Wall** - $1.98\n2. **Warner 25 Anos** - $1.98\n3. **Plays Metallica By Four Cellos** - $1.98\n4. **The Best Of Buddy Guy - The Millenium Collection** - $1.98\n5. **Carnaval 2001** - $1.98\n6. **The Essential Miles Davis [Disc 1]** - $1.98\n7. **The Battle Rages On** - $1.98\n8. **Roda De Funk** - $1.98\n9. **Demorou...** - $1.98\n10. **Motley Crue Greatest Hits** - $1.98\n\nAll these albums have a total price of $1.98 based on their sales.",
            additional_kwargs={"refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 192, "prompt_tokens": 1803, "total_tokens": 1995, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1536}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "stop", "logprobs": None},
            id="run-c525515a-cb55-48f7-94b2-0746439137f2-0",
            usage_metadata={"input_tokens": 1803, "output_tokens": 192, "total_tokens": 1995, "input_token_details": {"audio": 0, "cache_read": 1536}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
    ]
}

In [27]:
response["messages"][0]

HumanMessage(content='What are the 10 most expensive albums', additional_kwargs={}, response_metadata={}, id='08c293f3-c850-4861-bbae-ab333ea61dfc')

In [28]:
from langchain_core.output_parsers.json import JsonOutputParser

parser = JsonOutputParser()

parser.invoke(response)

ValidationError: 1 validation error for Generation
text
  Input should be a valid string [type=string_type, input_value={'messages': [HumanMessag...: 0, 'reasoning': 0}})]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type

In [2]:
reversed_messages = reversed(response["messages"])
answer_msg = next((msg for msg in reversed_messages if isinstance(msg, AIMessage)), None)
answer = answer_msg.content if answer_msg else ""


data_msg = next((msg for msg in reversed_messages if isinstance(msg, ToolMessage) and msg.name == "sql_db_query"), None)
data = data_msg.content if data_msg else ""
query_call_id = data_msg.tool_call_id if data_msg else ""


query_msg = next((msg for msg in reversed_messages if isinstance(msg, AIMessage) and msg.tool_calls and any(call["id"] == query_call_id for call in msg.tool_calls)), None)
query = query_msg.tool_calls[0]["args"]["query"] if query_msg else ""

print(f"Answer: {answer}")
print(f"Data: {data}")
print(f"Query: {query}")

Answer: The 10 most expensive albums based on total sales are:

1. **Balls to the Wall** - $1.98
2. **Warner 25 Anos** - $1.98
3. **Plays Metallica By Four Cellos** - $1.98
4. **The Best Of Buddy Guy - The Millenium Collection** - $1.98
5. **Carnaval 2001** - $1.98
6. **The Essential Miles Davis [Disc 1]** - $1.98
7. **The Battle Rages On** - $1.98
8. **Roda De Funk** - $1.98
9. **Demorou...** - $1.98
10. **Motley Crue Greatest Hits** - $1.98

All these albums have a total price of $1.98 based on their sales.
Data: [('Balls to the Wall', 1.98), ('Warner 25 Anos', 1.98), ('Plays Metallica By Four Cellos', 1.98), ('The Best Of Buddy Guy - The Millenium Collection', 1.98), ('Carnaval 2001', 1.98), ('The Essential Miles Davis [Disc 1]', 1.98), ('The Battle Rages On', 1.98), ('Roda De Funk', 1.98), ('Demorou...', 1.98), ('Motley Crue Greatest Hits', 1.98)]
Query: SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice 
FROM Album a 
JOIN InvoiceLine il ON a.AlbumId = il.TrackId 
GROUP

In [ ]:
reversed_response = [
    AIMessage(
        content="The 10 most expensive albums based on total sales are:\n\n1. **Balls to the Wall** - $1.98\n2. **Warner 25 Anos** - $1.98\n3. **Plays Metallica By Four Cellos** - $1.98\n4. **The Best Of Buddy Guy - The Millenium Collection** - $1.98\n5. **Carnaval 2001** - $1.98\n6. **The Essential Miles Davis [Disc 1]** - $1.98\n7. **The Battle Rages On** - $1.98\n8. **Roda De Funk** - $1.98\n9. **Demorou...** - $1.98\n10. **Motley Crue Greatest Hits** - $1.98\n\nAll these albums have a total price of $1.98 based on their sales.",
        additional_kwargs={"refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 192, "prompt_tokens": 1803, "total_tokens": 1995, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1536}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "stop", "logprobs": None},
        id="run-c525515a-cb55-48f7-94b2-0746439137f2-0",
        usage_metadata={"input_tokens": 1803, "output_tokens": 192, "total_tokens": 1995, "input_token_details": {"audio": 0, "cache_read": 1536}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content="[('Balls to the Wall', 1.98), ('Warner 25 Anos', 1.98), ('Plays Metallica By Four Cellos', 1.98), ('The Best Of Buddy Guy - The Millenium Collection', 1.98), ('Carnaval 2001', 1.98), ('The Essential Miles Davis [Disc 1]', 1.98), ('The Battle Rages On', 1.98), ('Roda De Funk', 1.98), ('Demorou...', 1.98), ('Motley Crue Greatest Hits', 1.98)]", name="sql_db_query", id="70d28606-e327-48bb-927c-d48eff42b663", tool_call_id="call_Uyoj19aI1O4Zdrv705wIGYEg"),
    AIMessage(
        content="",
        additional_kwargs={"tool_calls": [{"id": "call_Uyoj19aI1O4Zdrv705wIGYEg", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query"}, "type": "function"}], "refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 73, "prompt_tokens": 1590, "total_tokens": 1663, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1408}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
        id="run-eec068c0-f712-4c08-9fb3-1619b1eb6120-0",
        tool_calls=[{"name": "sql_db_query", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_Uyoj19aI1O4Zdrv705wIGYEg", "type": "tool_call"}],
        usage_metadata={"input_tokens": 1590, "output_tokens": 73, "total_tokens": 1663, "input_token_details": {"audio": 0, "cache_read": 1408}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="da5403a8-0956-4342-93c1-346db7a38f8a", tool_call_id="call_vFl1B4aUK7JT3Hiup2VuvYEE"),
    AIMessage(
        content="",
        additional_kwargs={"tool_calls": [{"id": "call_vFl1B4aUK7JT3Hiup2VuvYEE", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 1447, "total_tokens": 1521, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1280}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
        id="run-8636c183-a2e7-4655-9fba-0ebf41abfe4b-0",
        tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_vFl1B4aUK7JT3Hiup2VuvYEE", "type": "tool_call"}],
        usage_metadata={"input_tokens": 1447, "output_tokens": 74, "total_tokens": 1521, "input_token_details": {"audio": 0, "cache_read": 1280}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="e7a0ba29-540e-44b4-b15b-8a518db40a0e", tool_call_id="call_3Zhdw8s6eUKZdqTSFJPdaday"),
    AIMessage(
        content="",
        additional_kwargs={"tool_calls": [{"id": "call_3Zhdw8s6eUKZdqTSFJPdaday", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 1304, "total_tokens": 1378, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1024}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
        id="run-2587b203-5208-4361-93bd-b184ae9da2d4-0",
        tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_3Zhdw8s6eUKZdqTSFJPdaday", "type": "tool_call"}],
        usage_metadata={"input_tokens": 1304, "output_tokens": 74, "total_tokens": 1378, "input_token_details": {"audio": 0, "cache_read": 1024}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content="Error: (sqlite3.OperationalError) no such column: il.AlbumId\n[SQL: SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)", name="sql_db_query", id="93c01a0d-4938-4e83-8923-622829d9f28c", tool_call_id="call_An4AdvOZFahBfIs0bPr2ilEL"),
    AIMessage(
        content="",
        additional_kwargs={"tool_calls": [{"id": "call_An4AdvOZFahBfIs0bPr2ilEL", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query"}, "type": "function"}], "refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 1123, "total_tokens": 1197, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
        id="run-8dd504f0-5099-45a5-85a9-d4e493b2c6be-0",
        tool_calls=[{"name": "sql_db_query", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_An4AdvOZFahBfIs0bPr2ilEL", "type": "tool_call"}],
        usage_metadata={"input_tokens": 1123, "output_tokens": 74, "total_tokens": 1197, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="728441e9-da64-4429-aaaa-283e8fad790f", tool_call_id="call_T8fcCUSzvSrq1xkeiMnW1SAp"),
    AIMessage(
        content="",
        additional_kwargs={"tool_calls": [{"id": "call_T8fcCUSzvSrq1xkeiMnW1SAp", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 980, "total_tokens": 1054, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
        id="run-084ef297-ca7b-41b8-9414-dff3295042fd-0",
        tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_T8fcCUSzvSrq1xkeiMnW1SAp", "type": "tool_call"}],
        usage_metadata={"input_tokens": 980, "output_tokens": 74, "total_tokens": 1054, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content='\nCREATE TABLE "InvoiceLine" (\n\t"InvoiceLineId" INTEGER NOT NULL, \n\t"InvoiceId" INTEGER NOT NULL, \n\t"TrackId" INTEGER NOT NULL, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\t"Quantity" INTEGER NOT NULL, \n\tPRIMARY KEY ("InvoiceLineId"), \n\tFOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), \n\tFOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")\n)\n\n/*\n3 rows from InvoiceLine table:\nInvoiceLineId\tInvoiceId\tTrackId\tUnitPrice\tQuantity\n1\t1\t2\t0.99\t1\n2\t1\t4\t0.99\t1\n3\t2\t6\t0.99\t1\n*/', name="sql_db_schema", id="44e0561a-fa3a-4454-a33d-ebd433e53ab8", tool_call_id="call_8br8wAw8mKZJrolWl8xnI3yn"),
    ToolMessage(content='\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/', name="sql_db_schema", id="b4d20b87-a5d6-4fa7-b586-6eef045a68d4", tool_call_id="call_FkDZxE17O28rtluSfsePh1vy"),
    AIMessage(
        content="",
        additional_kwargs={"tool_calls": [{"id": "call_FkDZxE17O28rtluSfsePh1vy", "function": {"arguments": '{"table_names": "Album"}', "name": "sql_db_schema"}, "type": "function"}, {"id": "call_8br8wAw8mKZJrolWl8xnI3yn", "function": {"arguments": '{"table_names": "InvoiceLine"}', "name": "sql_db_schema"}, "type": "function"}], "refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 49, "prompt_tokens": 600, "total_tokens": 649, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
        id="run-3b412371-8877-42c3-a5b3-0be49942be52-0",
        tool_calls=[{"name": "sql_db_schema", "args": {"table_names": "Album"}, "id": "call_FkDZxE17O28rtluSfsePh1vy", "type": "tool_call"}, {"name": "sql_db_schema", "args": {"table_names": "InvoiceLine"}, "id": "call_8br8wAw8mKZJrolWl8xnI3yn", "type": "tool_call"}],
        usage_metadata={"input_tokens": 600, "output_tokens": 49, "total_tokens": 649, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content="Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track", name="sql_db_list_tables", id="2639cd20-79be-4145-8c28-fd7c98c11998", tool_call_id="call_ujwhC8lXKlibq8mdGbL3hOQo"),
    AIMessage(content="", additional_kwargs={"tool_calls": [{"id": "call_ujwhC8lXKlibq8mdGbL3hOQo", "function": {"arguments": "{}", "name": "sql_db_list_tables"}, "type": "function"}], "refusal": None}, response_metadata={"token_usage": {"completion_tokens": 12, "prompt_tokens": 554, "total_tokens": 566, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None}, id="run-00fb77df-ea09-45a7-9c04-075dc6692af4-0", tool_calls=[{"name": "sql_db_list_tables", "args": {}, "id": "call_ujwhC8lXKlibq8mdGbL3hOQo", "type": "tool_call"}], usage_metadata={"input_tokens": 554, "output_tokens": 12, "total_tokens": 566, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}}),
    HumanMessage(content="What are the 10 most expensive albums", additional_kwargs={}, response_metadata={}, id="08c293f3-c850-4861-bbae-ab333ea61dfc"),
]

In [17]:
answer = next((msg.content for msg in reversed_messages if isinstance(msg, int)), "")
print(answer)

In [5]:
response["messages"] = []

In [6]:
result = next(msg for msg in reversed(response["messages"]) if isinstance(msg, ToolMessage) and msg.name == "sql_db_query")
print(result.content)

StopIteration: 

In [14]:
query = next(msg for msg in reversed(response["messages"]) if isinstance(msg, ToolMessage) and msg.name == "sql_db_query_checker")
print(query.content)

```sql
SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice 
FROM Album a 
JOIN InvoiceLine il ON a.AlbumId = il.TrackId 
GROUP BY a.AlbumId, a.Title 
ORDER BY TotalPrice DESC 
LIMIT 10;
```
